Este notebook está vinculado al caso de estudio "Yapo" (ver el [documento](https://docs.google.com/document/d/1SwhwMOyG7-cmzasDeisQZ_NzX5v2VoZ7--lI4UhJz_U/edit#heading=h.5qaqs0frqkx4) de presentación). Se busca construir una base de datos de anuncios en Los Rios recopilando algunos anuncios de Yapo.cl.



## Modelo Entidad-Relacion

* Anuncio([url], titulo, descripcion, precio, moneda, fecha_publicacion, categoria, #id_vendedor))
* Vendedor([id_vendedor], nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion)



## Modelo Documento
```
Colección "Anuncios":
{
    titulo:
    descripcion:
    precio:
    fecha:
    url:
    categoria:
    user:
    {
        nombre:
        telefono:
        ciudad:
    }
}
```


##  Creacion de la base de datos y sus tablas


In [13]:
import mysql.connector

password = ""
db_connection = mysql.connector.connect(user="root",host="localhost",password=password)
cursor = db_connection.cursor()

In [ ]:
try:
    cursor.execute("DROP DATABASE Yapo;")
except:
    print("db doesnt exit")
    
cursor.execute("CREATE DATABASE Yapo;")
cursor.execute("USE Yapo")

#tabla vendedor
cursor.execute("CREATE TABLE vendedor ("+
               "id_vendedor VARCHAR(130) PRIMARY KEY, nombre VARCHAR(100), ciudad VARCHAR(30), codigo_region VARCHAR(7), "+
               "nombre_region VARCHAR(30), "+
               "fecha_inscripcion DATE)")

#tabla anuncio
cursor.execute("CREATE TABLE anuncio (url VARCHAR(300) PRIMARY KEY, "+
               "titulo VARCHAR(200), descripcion MEDIUMTEXT, precio INT, moneda VARCHAR(2), fecha_publicacion DATE, categoria VARCHAR(50), "
               +"id_vendedor VARCHAR(130), FOREIGN KEY (id_vendedor) REFERENCES vendedor(id_vendedor))")

## Llenado de la base de datos

In [6]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
from datetime import datetime, date

import datetime

def convert_date(yapo_date):
    month=yapo_date.split(" ")[0]
    year=yapo_date.split(" ")[1]
    
    convert = {'Enero' : 1,
            'Febrero' : 2,
            'Marzo' : 3,
            'Abril' : 4,
            'Mayo' : 5,
            'Junio' : 6,
            'Julio' : 7,
            'Agosto' : 8,
            'Septiembre' : 9, 
            'Octubre' : 10,
            'Noviembre' : 11,
            'Diciembre' : 12
           }
    
    
    new_date=datetime.date(int(year), convert[month], 1) 

    return new_date

print(convert_date("Febrero 2020"))

2020-02-01


In [7]:
browser = webdriver.Firefox() #usar tu geckodriver instalado

myUrl = 'https://www.yapo.cl/chile/todos_los_avisos?ca=12_s&l=0'
#myUrl = 'https://www.yapo.cl/los_rios/todos_los_avisos?ca=11_s&l=0&w=1&cmn=243'

browser.get(myUrl)
pageSoup = soup(browser.page_source, 'html.parser')
pages = pageSoup.find('span',  {'class', 'nohistory FloatRight'}).a['href']
index = pages.rfind('=')
lastPage = int(pages[index+1:])
root_pages = pages[:index+1]

In [9]:
for i in range(lastPage):
    
    #recuperarmos la URL de la pagina corriente
    url = root_pages + str(i+1)
    
    #con Selenium, vamos en navegar en esta página
    browser.get(url)
    
    #empezamos el scrapping de la pagina corriente
    pageSoup = soup(browser.page_source, 'html.parser')
    
    #recuperamos todos los tags HTML que corresponden a la lista de anuncios en esta pagina
    links = pageSoup.findAll('td', {'class' : 'thumbs_subject'})
    
    #empezamos a iterar sobre cada anuncio
    for link in links:
        
        #todos los datos que necesitamos encontrar
        url, titulo, descripcion, precio, fecha_publicacion, categoria="","","",None,"", ""
        nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion="","","","",""
        
        #Navegamos hacia la pagina del anuncio
        url=link.find('a',{'class':'title'})['href']
        print(url)
        browser.get(link.find('a',{'class':'title'})['href'])

        #RECUPERAMOS LA FECHA DE PUBLICACION DEL ANUNCIO          
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('time')):
            datetime_raw = pageSoup.find('time').attrs['datetime']
            date_publication_raw=datetime_raw.split("T")[0]
            date_publication_raw=  date_publication_raw.split("-")
            
            date_publication=datetime.date(int(date_publication_raw[0]), int(date_publication_raw[1]),
                                           int(date_publication_raw[2]))
            print("datetime:")
            print(date_publication)
        
        #RECUPERAMOS EL TITULO DEL ANUNCIO           
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('h1', {"id" : "da_subject"})):
            titulo = pageSoup.find('h1', {"id" : "da_subject"}).text.strip()
            print(titulo)
            
        #RECUPERAMOS LA DESCRIPCION DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "description"})):
            try:
                descripcion = pageSoup.find('div', {"class" : "description"}).text.split(' ', 1)[1].strip().replace(u'\n', u' ')
            except:
                continue
                
        #RECUPERAMOS EL PRECIO DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "price text-right"})):
            precio_raw = pageSoup.find('div', {"class" : "price text-right"}).text.strip().replace(u'\n', u' ').replace(u'\t', u'')
            precio_raw = precio_raw.split(" ")
            moneda = precio_raw[0]
            precio = precio_raw[1].split(",")[0].replace(u'.', u'')
            print("precio:"+precio)
            
        #RECUPERAMOS LA CATEGORIA DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "breadcrumbs"})):
            categoria = pageSoup.find('div', {"class" : "breadcrumbs"}).find('a', {"id" : "breadcrumb_category"}).find('strong').text.strip().replace(u'\n', u' ')
            print(categoria)
            
        
        #RECUPERAMOS EL NOMBRE DEL VENDEDOR, SU FECHA DE INSCRIPCION Y SU LOCALIDAD
        if(pageSoup.find('aside', {"class" : "sidebar-right"})):
            aside = pageSoup.find('aside', {"class" : "sidebar-right"})
    
            #print(aside.find('seller-info'))
            
            #NOMBRE
            if(aside.find('seller-info')!=None):
                nombre=aside.find('seller-info').attrs['username']
            
                #FECHA DE INSCRIPCION
                fecha_inscripcion_raw=aside.find('seller-info').attrs['seniority']
                try:
                    fecha_inscripcion_raw=fecha_inscripcion_raw[len("En Yapo desde "):]
                    fecha_inscripcion=convert_date(fecha_inscripcion_raw)
                    print(fecha_inscripcion)
                except:
                    fecha_inscripcion=None
                    continue
                
                #LOCALIDAD
                localidad_raw=aside.find('seller-info').attrs['region']
            
                #print(localidad_raw)
            
                region_raw=localidad_raw.split(",")[0]
                ciudad_raw=localidad_raw.split(",")[1]
            
                codigo_region=region_raw.split(" ")[0]
                nombre_region=region_raw[len(codigo_region)+1:]
                ciudad=ciudad_raw[1:]
                
        # LLENAMOS LA BASE DE DATOS: TABLA VENDEDOR
        
        try:
            sql = "INSERT INTO vendedor (id_vendedor,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion) VALUES (%s,%s, %s, %s,%s,%s)"
            val = (nombre+"_"+codigo_region+"_"+ciudad,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion)
            cursor.execute(sql, val)
        except Exception as e1:
            print(e1)
            #continue
        
        # LLENAMOS LA BASE DE DATOS: TABLA ANUNCIO
        try:
            sql = "INSERT INTO anuncio (url, titulo, descripcion, precio, moneda, fecha_publicacion, categoria, id_vendedor) VALUES (%s, %s, %s,%s, %s, %s, %s, %s)"
            val = (url, titulo, descripcion, int(precio), moneda, date_publication, categoria, nombre+"_"+codigo_region+"_"+ciudad)
            cursor.execute(sql, val)
        except Exception as e2:
            print(e2)
            #continue
        
        cursor.execute("COMMIT")

https://www.yapo.cl/region_metropolitana/electrodomesticos/lavadora_samsung__74033930.htm?ca=15_s&first=1&oa=74033930&xsp=0
datetime:
2020-08-29
Lavadora Samsung
precio:100000
Electrodomésticos
2020-08-01
https://www.yapo.cl/magallanes_antartica/accesorios_vehiculos/parrilla_porter_74033929.htm?ca=14_s&oa=74033929&xsp=1
datetime:
2020-08-29
Parrilla porter
precio:50000
Accesorios y piezas para vehículos
2019-09-01
https://www.yapo.cl/region_metropolitana/muebles/silla_reclinable_masajeadora_74033928.htm?ca=15_s&oa=74033928&xsp=2
datetime:
2020-08-29
Silla reclinable masajeadora
precio:135000
Muebles
2017-04-01
https://www.yapo.cl/region_metropolitana/moda-vestuario/jeans_levis__tenido_original__74033927.htm?ca=15_s&oa=74033927&xsp=3
datetime:
2020-08-29
Jeans levis (teñido original)
precio:10000
Moda y vestuario
2020-02-01
https://www.yapo.cl/biobio/moda-vestuario/chaqueta_stussy_raincoat_74033926.htm?ca=9_s&oa=74033926&xsp=4
datetime:
2020-08-29
Chaqueta Stussy Raincoat
precio:60000
M

datetime:
2020-08-29
Iphone 6 plus
precio:55000
Celulares, teléfonos y accesorios
2018-07-01
https://www.yapo.cl/coquimbo/ofertas_de_empleo/seguridad_73873975.htm?ca=5_s&oa=73873975&xsp=39
datetime:
2020-08-29
Seguridad
precio:380000
Ofertas de empleo
2014-11-01
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/soporte_para_hyundai_accent_ano_98_bnn_74033894.htm?ca=15_s&oa=74033894&xsp=40
datetime:
2020-08-29
Soporte para Hyundai accent año 98 bnn
precio:15000
Accesorios y piezas para vehículos
2020-08-01
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/biela_con_pistones_pa_hyundai_accent_prime_1_5_bnn_74033893.htm?ca=15_s&oa=74033893&xsp=41
datetime:
2020-08-29
Biela con pistones pa Hyundai accent prime 1.5 bnn
precio:50000
Accesorios y piezas para vehículos
2020-08-01
1062 (23000): Duplicate entry 'DesarmaduriaSotoMayor2_Región_Conchalí' for key 'vendedor.PRIMARY'
https://www.yapo.cl/tarapaca/articulos-del-hogar/cubre_cama_king_74033892.htm?ca=2_s&oa=7403389

datetime:
2020-08-29
Mantencion y reparacion
precio:20000
Servicios
https://www.yapo.cl/region_metropolitana/television_camaras/audifonos_inalambricos_klipxtreme_tranz_74033863.htm?ca=15_s&oa=74033863&xsp=22
datetime:
2020-08-29
Audífonos inalámbricos KlipXtreme Tranz
precio:73000
Audio, TV, video y fotografía
2020-04-01
https://www.yapo.cl/antofagasta/accesorios_vehiculos/bidon_metalico_hermetico_20_litro_74033862.htm?ca=3_s&oa=74033862&xsp=23
datetime:
2020-08-29
Bidón metálico hermético 20 litro
precio:40000
Accesorios y piezas para vehículos
2020-04-01
https://www.yapo.cl/region_metropolitana/moda-vestuario/cardigan_tejido__74033861.htm?ca=15_s&oa=74033861&xsp=24
datetime:
2020-08-29
Cardigan Tejido
precio:3990
Moda y vestuario
2019-06-01
https://www.yapo.cl/region_metropolitana/electrodomesticos/refrigerador_74033860.htm?ca=15_s&oa=74033860&xsp=25
datetime:
2020-08-29
Refrigerador
precio:50000
Electrodomésticos
2019-02-01
https://www.yapo.cl/region_metropolitana/salud-belleza/masc

datetime:
2020-08-29
Casa campo
precio:80000000
Vendo - Casa
2020-08-01
https://www.yapo.cl/region_metropolitana/salud-belleza/cubre_calzado_desechable_73913500.htm?ca=15_s&oa=73913500&xsp=9
datetime:
2020-08-29
CUBRE CALZADO desechable
Salud y belleza
2020-06-01
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/los_lagos/animales_accesorios/hermosos_bramas_73911402.htm?ca=12_s&oa=73911402&xsp=10
datetime:
2020-08-29
Hermosos Bramas
precio:35000
Animales y sus accesorios
https://www.yapo.cl/arica_parinacota/accesorios_vehiculos/batea_random_liberada_2011_ano_original_2_1_73829448.htm?ca=1_s&oa=73829448&xsp=11
datetime:
2020-08-29
Batea random liberada 2011 año original 2+1
precio:6800000
Accesorios y piezas para vehículos
2014-05-01
https://www.yapo.cl/los_lagos/comprar/2_cabanas_en_medio_de_un_bosque_en_puerto_varas_72445502.htm?ca=12_s&oa=72445502&xsp=12
datetime:
2020-08-29
2 cabañas en medio de un bosque,en Puerto Varas
precio:7000

datetime:
2020-08-29
HERMOSO DEPARTAMENTo, 2 Dormitorios
precio:248000
Arriendo - Departamento
2015-08-01
https://www.yapo.cl/los_lagos/moda-vestuario/polar_lippi_talla_m__74033815.htm?ca=12_s&oa=74033815&xsp=46
datetime:
2020-08-29
Polar Lippi talla M
precio:5000
Moda y vestuario
2020-08-01
https://www.yapo.cl/ohiggins/computadores/procesador___ssd_74033814.htm?ca=7_s&oa=74033814&xsp=47
datetime:
2020-08-29
Procesador + SSD
precio:34990
Computadores y accesorios
2017-09-01
https://www.yapo.cl/region_metropolitana/negocios_maquinaria_construccion/alicates_de_anillo_de_retencion_knipex_74033813.htm?ca=15_s&oa=74033813&xsp=48
datetime:
2020-08-29
Alicates de anillo de retención knipex
precio:27000
Negocios, maquinaria y construcción
2020-08-01
1062 (23000): Duplicate entry 'Denise BM_Región_Las Condes' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/electrodomesticos/lavadora_electrolux_74033812.htm?ca=15_s&oa=74033812&xsp=49
datetime:
2020-08-29
Lavadora Electrolux
p

...

datetime:
2020-08-29
Botas
precio:38500
Calzado
2014-11-01
1062 (23000): Duplicate entry 'Gastón_XV_Arica' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/moda-vestuario/marmot_precip_talla_s_74033487.htm?ca=15_s&oa=74033487&xsp=20
datetime:
2020-08-29
Marmot Precip talla S
precio:25000
Moda y vestuario
2020-08-01
1062 (23000): Duplicate entry 'Magdalena Duhalde_Región_Lo Barnechea' for key 'vendedor.PRIMARY'
https://www.yapo.cl/magallanes_antartica/juguetes/juego_interactivo_oruga_bailarina_74033486.htm?ca=14_s&oa=74033486&xsp=21
datetime:
2020-08-29
Juego Interactivo ORUGA BAILARINA
precio:8000
Juguetes
2020-06-01
1062 (23000): Duplicate entry 'Vendedor_XII_Punta Arenas' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/calzado/zapatos_escolares__74033485.htm?ca=15_s&oa=74033485&xsp=22
datetime:
2020-08-29
Zapatos escolares
precio:5000
Calzado
2018-11-01
1062 (23000): Duplicate entry 'Maria Rosangela Sanchez Hernandez _Región_Santiago' for key 'v

datetime:
2020-08-29
Profesor de matemáticas
Servicios
2018-04-01
1062 (23000): Duplicate entry 'johnf.empresa_II_Antofagasta' for key 'vendedor.PRIMARY'
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
https://www.yapo.cl/region_metropolitana/television_camaras/camara_fotografica_sony_73768338.htm?ca=15_s&oa=73768338&xsp=48
datetime:
2020-08-29
Camara fotográfica Sony
precio:30000
Audio, TV, video y fotografía
2014-12-01
1062 (23000): Duplicate entry 'manuel salinas_Región_Maipú' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/articulos-del-hogar/mallas_antimosquitos_para_ventana__74033458.htm?ca=15_s&oa=74033458&xsp=49
datetime:
2020-08-29
Mallas Antimosquitos Para Ventana
precio:7990
Otros artículos del hogar
2018-06-01
1062 (23000): Duplicate entry 'andrea jorquera_Región_Santiago' for key 'vendedor.PRIMARY'
https://www.yapo.cl/valparaiso/electrodomesticos/combo_lavadora___cocina_74033457.htm?ca=6_s&oa=74033457&xsp=0
datetime:
202

datetime:
2020-08-29
Pantalla-Monitor para ordenador, marca OLIDATA
precio:5000
Computadores y accesorios
2017-03-01
1062 (23000): Duplicate entry 'Ignacio_VIII_Concepción' for key 'vendedor.PRIMARY'
https://www.yapo.cl/arica_parinacota/vestuario-futura-mama-ninos/talco_simonds_natural_74033433.htm?ca=1_s&oa=74033433&xsp=26
datetime:
2020-08-29
Talco simonds natural
precio:2000
Vestuario futura mamá y niños
2017-12-01
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/culata_para_honda_accord_ano_98_std_bnn_74033432.htm?ca=15_s&oa=74033432&xsp=27
datetime:
2020-08-29
Culata para Honda accord año 98 std bnn
precio:70000
Accesorios y piezas para vehículos
2020-08-01
1062 (23000): Duplicate entry 'DesarmaduriaSotoMayor2_Región_Conchalí' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/articulos-del-hogar/fuente_tramontina_74033431.htm?ca=15_s&oa=74033431&xsp=28
datetime:
2020-08-29
Fuente Tramontina
precio:17000
Otros artículos del hogar
2020-03-01
https://ww

datetime:
2020-08-29
Nissan Pathfinder 3.0 1989
precio:1980000
Autos, camionetas y 4x4
2019-07-01
1062 (23000): Duplicate entry 'CQuero Automotriz_Región_La Florida' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/negocios_maquinaria_construccion/carbon_de_espino_30k__despacho_gratis_zona_oriente_74033405.htm?ca=15_s&oa=74033405&xsp=5
datetime:
2020-08-29
CARBÓN DE ESPINO 30K. Despacho gratis zona oriente
precio:22000
Negocios, maquinaria y construcción
2017-06-01
https://www.yapo.cl/region_metropolitana/articulos-del-hogar/sabanas__74033404.htm?ca=15_s&oa=74033404&xsp=6
datetime:
2020-08-29
Sabanas
precio:12990
Otros artículos del hogar
2017-11-01
1062 (23000): Duplicate entry 'Anais castro_Región_Estación Central' for key 'vendedor.PRIMARY'
https://www.yapo.cl/region_metropolitana/accesorios_vehiculos/culata_para_volkswagen_golf_1_8_ano_97_bnn_74033403.htm?ca=15_s&oa=74033403&xsp=7
datetime:
2020-08-29
Culata para Volkswagen golf 1.8 año 97 bnn
precio:80000
Acceso

KeyboardInterrupt: 

In [1]:
##5.1
sql = "select categoria, avg(precio) from anuncio where moneda = \"$\" group by categoria order by avg(precio) desc;"
cursor.execute(sql)
##5.2
sql = "SELECT nombre, count(*)/7 FROM anuncio JOIN vendedor USING (id_vendedor) WHERE fecha_publicacion BETWEEN \"2020-08-12\" AND \"2020-08-18\" GROUP BY nombre ORDER BY count(*) DESC;"
cursor.execute(sql)
##5.3
sql = "alter table vendedor add perfil varchar(20) default \"personal\";"
cursor.execute(sql)
vendedor = DataFrame(cursor.fetchall())
sql = '''update vendedor as V inner join (SELECT id_vendedor, count(*) FROM anuncio JOIN vendedor USING (id_vendedor) group by nombre having count(*) > 4) as SUB_V on V.id_vendedor = SUB_V.id_vendedor set V.perfil1 = "Pro";'''
cursor.execute(sql)
##5.4
sql = "select dayofweek(fecha_publicacion) as dia, categoria, avg(precio) from anuncio where moneda=\"$\" group by day order by avg(precio) desc;"
cursor.execute(sql)

NameError: name 'cursor' is not defined